# Geometric Tracking Control of a Quadrotor UAV
Questo notebook implementa il traking controller esposto nel paper **"Geometric Tracking Control of a Quadrotor UAV on SE(3)" di Taeyoung Lee**

In [ ]:
import numpy as np

In [ ]:
m = 4.34
g = 9.81
J = np.diag([0.0820, 0.0845, 0.1377])
Jinv = np.linalg.inv(J)


In [ ]:
def hat(v):
    """Return the 3x3 hat matrix of a 3-vector v"""
    return np.array([[0, -v[2], v[1]],
                     [v[2], 0, -v[0]],
                     [-v[1], v[0], 0]], dtype=float)

def rodrigues_exp(omega):
    """Matrix exponential of hat(omega) using Rodrigues' formula.
    omega: 3-vector. returns 3x3 rotation matrix exp(hat(omega))."""
    theta = np.linalg.norm(omega)
    if theta < 1e-12:
        return np.eye(3) + hat(omega)
    k = omega / theta
    K = hat(k)
    return np.eye(3) + np.sin(theta) * K + (1 - np.cos(theta)) * (K @ K)


## Quadcopter model


In [ ]:
def quadcopter_model(x,u):  
    f = u['f']
    M = u['M']

    pos = x['x']
    v = x['v']
    R = x['R']
    omega = x['omega']

    #
    x_dot = v

    # Dinamica traslazionale
    v_dot = np.array([0,0,g]) - (f/m) * (R @ np.array([0,0,1]))
    #
    R_dot = R @ hat(omega)
    #
    omega_dot = Jinv @ (M - np.cross(omega, J @ omega))

    return {
            'dx':x_dot,
            'dv':v_dot,
            'dR':R_dot,
            'domega':omega_dot
        }

## Controller

In [ ]:
def trackingController():
    pass

## Integration

In [ ]:
def step_euler(x,u,dt):
    x_dot = quadcopter_model(x,u)

    dx = x_dot['dx']
    dv = x_dot['dv']
    dR = x_dot['dR']
    domega = x_dot['domega']

    x_new = x['x'] + dx * dt
    v_new = x['v'] + dv * dt

    R_inc = rodrigues_exp(x['omega']*dt)
    R_new = x['R'] @ R_inc

    omega_new = x['omega'] + domega * dt

    return {
        'x':x_new,
        'v':v_new,
        'R':R_new,
        'omega':omega_new
    }

## Simulation

In [ ]:
x0 = {
    'x':np.array([0,0,0]),
    'v':np.array([0,0,0]),
    'R':np.eye(3),
    'omega':np.array([0,0,0])
}
u0 = {
    'f':0,
    'M':np.array([0,0,0])
}

tf_s = 10 # t final step in second
dt = 0.004 # time step

t_s = np.arange(0, tf_s, dt)    #time step
nt_s = t_s.size                 #number of timestep

for i in range(1, nt_s):
    pass

## Visualization